In [7]:
'''
This code takes the filtered VF data and finds the number of 
unique gene mutations in Onno's vs healthy human genome data. 
'''

"\nThis code takes the filtered VF data and finds the number of \nunique gene mutations in Onno's vs healthy human genome data. \n"

In [8]:
import numpy as np
import pandas as pd 

In [9]:
genes = pd.read_csv('genes.txt', delimiter='\t')
#drop all random mutations 
genes=genes[:60270]

genes.chrom = genes.chrom.apply(lambda x: (x[3:5]))
genes.tail()

ono = pd.read_csv('mut_o_filt.txt', delimiter='\t')
#ono.head()


In [10]:
ono.head()

,CHROM,POS,REF,ALT,sample
0,1,13649,G,C,onno
1,1,13757,G,A,onno
2,1,136962,C,T,onno
3,1,139836,G,C,onno
4,1,16861372,T,C,onno


In [11]:
genes.head(5)

,bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,NM_001308203,1,+,66999251,67216822,67000041,67208778,22,"66999251,66999928,67091529,67098752,67105459,6...","66999355,67000051,67091593,67098777,67105516,6...",0,SGIP1,cmpl,cmpl,"-1,0,1,2,0,0,1,0,1,2,1,1,1,0,1,1,2,2,0,2,1,1,"
1,0,NM_032291,1,+,66999638,67216822,67000041,67208778,25,"66999638,67091529,67098752,67101626,67105459,6...","67000051,67091593,67098777,67101698,67105516,6...",0,SGIP1,cmpl,cmpl,"0,1,2,0,0,0,1,0,0,0,1,2,1,1,1,1,0,1,1,2,2,0,2,..."
2,1,NM_001145277,1,+,16767166,16786584,16767256,16785491,7,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,"
3,1,NM_001145278,1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767270,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"
4,1,NM_018090,1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"


In [12]:
len(list(set(list(genes.name2.values)))) #how many genes total?

27273

In [ ]:
mut_index = []
for j, mutation in enumerate(range(ono.shape[0])):
    chrom = ono.CHROM[j]
    position = ono.POS[j]
    for i in range(genes.shape[0]):   

        try: 
            if int(chrom) == int(genes.chrom[i]): #ensure same chromosome 
                if position > genes.cdsStart[i] and position <genes.cdsEnd[i]: #if pos is within gene range 
                    mut_index.append(genes.name2[i])
                    print chrom, position, genes.cdsStart[i], genes.cdsEnd[i]
        except: 
            pass
                

In [ ]:
# turn into unique set 
mut_index_new = list(set(mut_index))

In [ ]:
# number of unique genes! 
len(mut_index_new)

In [ ]:
'''
This set of code is is used to filter the raw VCF data 
for cluster analysis 
'''

In [ ]:
#import the raw VCF files, save as df's

'''
CHROM: Number of chromosome on which variant is being called
POS: The 1- based position of the variation on the given sequence
ID: Identifier of the variation
REF: The referece base at the given position of the given reference sequence
ALT: List of alternative alleles at this position
QUAL: Quality score associated with the interference of the given alleles
GQ: Confidence that the assigned genotype is correct 

'''

if False: #toggle to prevent overprocessing... 

    with open('VCF_Data_Normal', 'rb') as f:
        df_normal = pickle.load(f)
    print (df_normal.head())

    with open('VCF_Data_Cancer', 'rb') as f:
        df_cancer = pickle.load(f)
    df_cancer.head()

In [ ]:
# Grab smaller subset of data for unit testing 

df_normal_test = df_normal#[:2000]
df_cancer_test = df_cancer#[:2000]

In [ ]:
#save for later 

df_normal_test.to_csv('normal.csv')
df_cancer_test.to_csv('cancer.csv')

In [ ]:
df_normal_test = pd.read_csv('normal.csv')
df_cancer_test = pd.read_csv('cancer.csv')

In [ ]:
#convert alternative lists into strings 

df_normal_test.ALT = df_normal_test.ALT.apply(lambda s: s[1:-1])#str(s[0]) if len(s)==1 else (str(s[0])+'*'))
df_cancer_test.ALT = df_cancer_test.ALT.apply(lambda s: s[1:-1])#str(s[0]) if len(s)==1 else (str(s[0])+'*'))

In [ ]:
# comapre the cancer (tumor) and healthy (blood) samples
# only keep rows where genes are different (mutations) 

df_normal_test['Test'] = df_normal_test['POS'].astype(str) + df_normal_test['REF'].astype(str) + df_normal_test['ALT'].astype(str) + df_normal_test['CHROM'].astype(str)
df_cancer_test['Test'] = df_cancer_test['POS'].astype(str) + df_cancer_test['REF'].astype(str) + df_cancer_test['ALT'].astype(str) + df_cancer_test['CHROM'].astype(str)

compare =pd.DataFrame(data = [df_cancer_test.Test, df_normal_test.Test]).T
compare.columns = ['cancer', 'normal']
compare.compare_cols = np.where((compare.cancer != compare.normal) )

In [ ]:
# import new filtered data (Natasha's)

VCF_filtered = pd.read_csv('VCF_filtered.csv')

In [ ]:
# How many poor GQ genes in dataset?

GQ_poor = []
for i in VCF_filtered.GQ.sort_values(ascending=True):
    if i< 80:
        GQ_poor.append(i)
print (len(GQ_poor))

In [ ]:
# drop rows where GQ is less than 95% confidence 

VCF_filtered = VCF_filtered.drop(VCF_filtered[VCF_filtered.GQ<95].index)
VCF_filtered
#VCF_filtered.to_csv('VCF_filtered_GQ_thresh.csv')

In [ ]:
VCF_filtered = VCF_filtered.sample(frac=600.0/len(VCF_filtered))

In [ ]:
VCF_filtered.to_csv('VCF_filtered_600.csv')